In [1]:
import wandb
import pandas as pd
from loguru import logger
from tqdm import tqdm
from itertools import product
import numpy as np
from pathlib import Path
import functools
import json
from typing import Literal

from histaug.utils import RunningStats, cached_df

api = wandb.Api()

INDEX_COLS = ["target", "train_dataset", "test_dataset", "model", "feature_extractor", "augmentations", "seed"]

RENAME_MODELS = {
    "AttentionMIL": "AttMIL",
    "MeanAveragePooling": "Mean pool",
    "Transformer": "Transformer",
}
RENAME_FEATURE_EXTRACTORS = {
    "bt": "Lunit-BT",
    "swav": "Lunit-SwAV",
    "dino_p16": "Lunit-DINO",
    "ctranspath": "CTransPath",
    "owkin": "Phikon",
    "resnet50": "ResNet-50",
    "retccl": "RetCCL",
    "swin": "Swin",
    "vit": "ViT",
}

RESULTS_DIR = Path("/app/results")

# Collect results from `wandb`

In [2]:
def filter_runs(runs, filters: dict):
    return [run for run in runs if all(getattr(run, key, None) == value for key, value in filters.items())]


def summarize_run(run):
    # return dict(
    #     target=(column := run.config["dataset"]["targets"][0]["column"]),
    #     train_dataset=run.config["dataset"]["name"],
    #     test_dataset=run.config["test"]["dataset"]["name"],
    #     model=run.config["model"]["_target_"].split(".")[-1],
    #     feature_extractor=run.config["settings"]["feature_extractor"],
    #     augmentations=run.config["dataset"]["augmentations"]["name"],
    #     seed=run.config["seed"],
    #     train_auroc=run.summary[f"train/{column}/auroc"]["best"],
    #     val_auroc=run.summary[f"val/{column}/auroc"]["best"],
    #     test_auroc=run.summary[f"test/{column}/auroc"]["best"],
    # )

    history = run.history().groupby("epoch").first()
    best = history[~history.index.isna()].sort_values("val/loss", ascending=True).iloc[0]
    column = run.config["dataset"]["targets"][0]["column"]
    if f"test/{column}/auroc" in run.summary:
        test_auroc = run.summary[f"test/{column}/auroc"]["best"]
    else:
        test_auroc = history[f"test/{column}/auroc"].max()
    return dict(
        target=column,
        train_dataset=run.config["dataset"]["name"],
        test_dataset=run.config["test"]["dataset"]["name"],
        model=run.config["model"]["_target_"].split(".")[-1],
        feature_extractor=run.config["settings"]["feature_extractor"],
        augmentations=run.config["dataset"]["augmentations"]["name"],
        seed=run.config["seed"],
        train_auroc=best[f"train/{column}/auroc"],
        val_auroc=best[f"val/{column}/auroc"],
        test_auroc=test_auroc,
    )


@cached_df(lambda: "aurocs")
def load_aurocs():
    logger.info("Loading runs")
    runs = list(api.runs("histaug"))
    runs = filter_runs(runs, {"state": "finished"})
    runs = [summarize_run(run) for run in tqdm(runs, desc="Loading run data")]
    runs = [run for run in runs if run is not None]
    df = pd.DataFrame(runs)
    df = df.set_index(INDEX_COLS).sort_index().drop_duplicates()
    return df


df = df_all = load_aurocs().drop_duplicates()
df = (
    df.reset_index()
    .groupby(["target", "train_dataset", "test_dataset", "model", "feature_extractor", "augmentations"])
    .filter(lambda x: sorted(x.seed.values) == list(range(5)))
    .set_index(df.index.names)
    .sort_index()
)
print("Removed runs:", len(df_all) - len(df))
print(
    df_all.index.difference(df.index)
    .to_frame(index=False)
    .groupby([x for x in df.index.names if x != "seed"])
    .seed.count()
)

2023-11-04 19:49:52.543 | INFO     | __main__:load_aurocs:42 - Loading runs
Loading run data: 100%|██████████| 3743/3743 [30:21<00:00,  2.05it/s]


Removed runs: 77
target  train_dataset  test_dataset    model               feature_extractor  augmentations    
BRAF    tcga_crc_BRAF  cptac_crc_BRAF  AttentionMIL        ctranspath         Macenko_patchwise    4
                                                                              Macenko_slidewise    2
                                                           dino_p16           Macenko_patchwise    4
                                                           owkin              Macenko_slidewise    2
                                                           resnet50           Macenko_patchwise    4
                                                           swav               Macenko_patchwise    4
                                                           swin               Macenko_patchwise    4
                                                           vit                Macenko_patchwise    4
                                                                              M

In [3]:
df = df.query("target in ['CDH1', 'PIK3CA', 'TP53', 'subtype', 'MSI', 'KRAS']")

# Compare original vs Macenko (vs Macenko_slidewise)

In [4]:
macenko = df.query("augmentations == 'Macenko_patchwise'")["test_auroc"].droplevel("augmentations")
# macenko = df.query("augmentations == 'Macenko_slidewise'")["test_auroc"].droplevel("augmentations")
orig = df.query("augmentations == 'none'")["test_auroc"].droplevel("augmentations")
# Mean diff across seeds
d = (
    (macenko - orig)
    .rename("test_auroc_diff")
    .reset_index()
    .drop(columns="seed")
    .groupby(["target", "train_dataset", "test_dataset", "model", "feature_extractor"])
    .agg(["mean", "std"])
)
o = (
    orig.rename("test_auroc_orig")
    .reset_index()
    .drop(columns="seed")
    .groupby(["target", "train_dataset", "test_dataset", "model", "feature_extractor"])
    .agg(["mean", "std"])
)
d = pd.concat([d, o], axis=1)
d.query("model == 'Transformer'")

test_auroc_diff  \
                                                                                      mean   
target  train_dataset     test_dataset       model       feature_extractor                   
CDH1    tcga_brca_CDH1    cptac_brca_CDH1    Transformer bt                       0.011136   
                                                         ctranspath              -0.035294   
                                                         dino_p16                 0.017247   
                                                         owkin                   -0.005882   
                                                         resnet50                -0.071673   
                                                         retccl                  -0.016562   
                                                         swav                     0.026271   
                                                         swin                    -0.070246   
                                                         vit                     -0.022559   
MSI     tcga_crc_MSI      cptac_crc_MSI      Transformer bt                      -0.008848   
                                                         ctranspath               0.003961   
                                                         dino_p16                 0.002984   
                                                         owkin                    0.027109   
                                                         resnet50                -0.001337   
                                                         retccl                   0.091770   
                                                         swav                     0.003292   
                                                         swin                     0.016872   
                                                         vit                     -0.016255   
PIK3CA  tcga_brca_PIK3CA  cptac_brca_PIK3CA  Transformer bt                       0.045800   
                                                         ctranspath               0.041057   
                                                         dino_p16                 0.006371   
                                                         owkin                   -0.002400   
                                                         resnet50                 0.017486   
                                                         retccl                   0.044771   
                                                         swav                     0.042086   
                                                         swin                     0.023743   
                                                         vit                      0.048714   
TP53    tcga_brca_TP53    cptac_brca_TP53    Transformer bt                      -0.031145   
                                                         ctranspath               0.000365   
                                                         dino_p16                -0.001712   
                                                         owkin                   -0.000281   
                                                         resnet50                -0.090376   
                                                         retccl                  -0.029405   
                                                         swav                     0.012177   
                                                         swin                     0.006173   
                                                         vit                     -0.009007   
subtype tcga_brca_subtype cptac_brca_subtype Transformer bt                      -0.002526   
                                                         ctranspath              -0.007959   
                                                         dino_p16                -0.005454   
                                                         owkin                    0.044769   
                                 

# What is the best feature extractor?

In [5]:
augmentation = "none"
augmentation = "Macenko_patchwise"
augmentation = "Macenko_slidewise"

test_aurocs = df.query(f"augmentations == '{augmentation}'").droplevel("augmentations")["test_auroc"]
# test_aurocs = df.query(f"augmentations == '{augmentation}' and target == 'MSI'").droplevel("augmentations")["test_auroc"]
test_aurocs

target   train_dataset      test_dataset        model         feature_extractor  seed
CDH1     tcga_brca_CDH1     cptac_brca_CDH1     AttentionMIL  bt                 0       0.793832
                                                                                 1       0.752142
                                                                                 2       0.737293
                                                                                 3       0.807539
                                                                                 4       0.727584
                                                                                           ...   
subtype  tcga_brca_subtype  cptac_brca_subtype  Transformer   vit                0       0.633009
                                                                                 1       0.706982
                                                                                 2       0.756052
                                

In [7]:
def compute_average_offset_from_best(sub_df):
    """Function to compute average offset from best for a given subset of data."""
    pivot_data = sub_df.pivot(index="seed", columns="feature_extractor", values="test_auroc")
    feature_extractors = pivot_data.columns.values
    seeds = pivot_data.index.values
    combinations = product(*pivot_data.values.T)
    n_combinations = int(len(seeds) ** len(feature_extractors))
    stats_by_feature_extractor = {fe: RunningStats() for fe in feature_extractors}

    for i, auroc_values in enumerate(tqdm(combinations, total=n_combinations)):
        # sorted_indices = np.argsort(auroc_values)[::-1]
        # ranks_array[sorted_indices] += np.arange(1, len(feature_extractors) + 1)
        diffs = np.array(auroc_values).max() - np.array(auroc_values)
        for fe, diff in zip(feature_extractors, diffs):
            stats_by_feature_extractor[fe].update(diff)

    return {fe: stats.compute() for fe, stats in stats_by_feature_extractor.items()}


@cached_df(lambda *args, **kwargs: f"offsets_from_best_{augmentation}")
def compute_results_table(test_aurocs: pd.Series):
    """Compute average offsets from best for each (target, model) pair."""
    d = test_aurocs.reset_index()

    results = {}
    unique_pairs = d[["target", "model"]].drop_duplicates().values
    unique_pairs = [("PIK3CA", "AttentionMIL")]

    for target, model in unique_pairs:
        sub_data = d[(d["target"] == target) & (d["model"] == model)]
        results[(target, model)] = compute_average_offset_from_best(sub_data)
        print(
            f"{target:10s} {model:20s}:",
            ", ".join(
                f"{k}={mean:.2f}+-{std:.2f}"
                for (k, (mean, std)) in sorted(results[(target, model)].items(), key=lambda x: x[1], reverse=False)
            ),
        )

    r = pd.DataFrame(results).map(lambda x: x._asdict())
    r.index.name = "feature_extractor"
    r.columns.names = ["target", "model"]
    r = r.stack().stack().apply(pd.Series)
    r.columns.names = ["stats"]
    r = (
        r.pivot_table(index=["model", "feature_extractor"], columns="target")
        .reorder_levels([1, 0], axis=1)
        .sort_index(axis=1)
    )
    return r


r = compute_results_table(test_aurocs)
r

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# Computer overall mean and std (across targets)
n_targets = r.stack().query("stats == 'mean'").droplevel("stats").shape[1]
overall_mean = r.stack().query("stats == 'mean'").droplevel("stats").sum(axis="columns").divide(n_targets)
overall_std = r.stack().query("stats == 'std'").droplevel("stats").pow(2).sum(axis="columns").pow(0.5).divide(n_targets)
r["average", "mean"] = overall_mean
r["average", "std"] = overall_std
r

target                                    CDH1              PIK3CA            \
stats                                     mean       std      mean       std   
model              feature_extractor                                           
AttentionMIL       bt                 0.040381  0.014883  0.131119  0.023538   
                   ctranspath         0.021078  0.029036  0.036662  0.013538   
                   dino_p16           0.009999  0.011961  0.016805  0.015374   
                   owkin              0.044607  0.013955  0.052319  0.018709   
                   resnet50           0.183557  0.036119  0.013376  0.011095   
                   retccl             0.034899  0.020714  0.023462  0.018311   
                   swav               0.026332  0.027243  0.051405  0.026158   
                   swin               0.196406  0.042599  0.045090  0.029057   
                   vit                0.115310  0.031582  0.041605  0.040126   
MeanAveragePooling bt                 0.021193  0.008257  0.050165  0.013826   
                   ctranspath         0.005202  0.006431  0.005823  0.008117   
                   dino_p16           0.016510  0.011786  0.034108  0.018017   
                   owkin              0.009086  0.009219  0.078023  0.036973   
                   resnet50           0.109714  0.044172  0.025823  0.012864   
                   retccl             0.016853  0.007492  0.026908  0.013157   
                   swav               0.019252  0.011039  0.113708  0.011704   
                   swin               0.124106  0.020555  0.008908  0.008221   
                   vit                0.087784  0.007742  0.028080  0.014527   
Transformer        bt                 0.040734  0.026102  0.074068  0.040549   
                   ctranspath         0.052042  0.036446  0.058554  0.041356   
                   dino_p16           0.048444  0.037183  0.030640  0.033469   
                   owkin              0.085166  0.031200  0.069240  0.035753   
                   resnet50           0.177170  0.099972  0.043497  0.054200   
                   retccl             0.057639  0.041600  0.062354  0.025768   
                   swav               0.013093  0.014527  0.083240  0.045065   
                   swin               0.187393  0.050929  0.089297  0.037696   
                   vit                0.127542  0.026785  0.080325  0.029667   

target                                    TP53             subtype            \
stats                                     mean       std      mean       std   
model              feature_extractor                                           
AttentionMIL       bt                 0.023166  0.023604  0.113091  0.040531   
                   ctranspath         0.028048  0.023851  0.001350  0.003717   
                   dino_p16           0.042863  0.026436  0.044301  0.032044   
                   owkin              0.088795  0.035863  0.109806  0.030453   
                   resnet50           0.167975  0.054579  0.171760  0.035164   
                   retccl             0.032341  0.038449  0.085243  0.039041   
                   swav               0.036241  0.017693  0.054317  0.028419   
                   swin               0.237223  0.028644  0.079680  0.030228   
                   vit                0.122184  0.017218  0.105321  0.039103   
MeanAveragePooling bt                 0.031026  0.027218  0.056000  0.034604   
                   ctranspath         0.018400  0.024852  0.000000  0.000000   
                   dino_p16           0.044887  0.044282  0.045113  0.009028   
                   owkin              0.114864  0.031651  0.109929  0.006369   
                   resnet50           0.074741  0.025101  0.075478  0.003895   
                   retccl             0.045925  0.029108  0.012317  0.003628   
                   swav               0.036048  0.024649  0.061725  0.003120   
                   swin               0.113013  0.039164  0.064694  0.008130   

In [7]:
def results_to_latex(r: pd.DataFrame, goal: Literal["min", "max"] = "min") -> str:
    # Format for appearance
    r = r.unstack("model")
    means = r.stack("stats").query("stats == 'mean'").droplevel("stats")
    stds = r.stack("stats").query("stats == 'std'").droplevel("stats")
    formatted = means.map(lambda x: f"{x:.2f}") + " \\pm " + stds.map(lambda x: f"{x:.2f}")

    # Make best model bold
    best_mask = means == getattr(means, goal)(axis="index")
    formatted[best_mask] = "\\mathbf{" + formatted[best_mask] + "}"
    formatted = "$" + formatted + "$"
    formatted = formatted.stack("model")
    formatted = formatted.swaplevel("feature_extractor", "model").sort_index()

    # Check if we have average column
    targets = formatted.columns.get_level_values("target").unique()
    has_average = "average" in targets
    if has_average:
        targets = targets.drop("average")

    # Rename
    formatted = formatted.rename(RENAME_MODELS, level="model")
    formatted = formatted.rename(RENAME_FEATURE_EXTRACTORS, level="feature_extractor")
    formatted.index.names = ["Model", "Feature extractor"]
    formatted.columns.names = ["Target"]
    return formatted.to_latex(
        column_format="l" * len(formatted.index.levels) + "|" + "c" * len(targets) + ("|c" if has_average else ""),
        escape=False,
    )


print(results_to_latex(r))

\begin{tabular}{ll|cccc|c}
\toprule
 & Target & CDH1 & PIK3CA & TP53 & subtype & average \\
Model & Feature extractor &  &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{AttMIL} & Lunit-BT & $0.04 \pm 0.01$ & $0.13 \pm 0.02$ & $\mathbf{0.02 \pm 0.02}$ & $0.11 \pm 0.04$ & $0.08 \pm 0.01$ \\
 & CTransPath & $0.02 \pm 0.03$ & $0.04 \pm 0.01$ & $0.03 \pm 0.02$ & $\mathbf{0.00 \pm 0.00}$ & $\mathbf{0.02 \pm 0.01}$ \\
 & Lunit-DINO & $\mathbf{0.01 \pm 0.01}$ & $0.02 \pm 0.02$ & $0.04 \pm 0.03$ & $0.04 \pm 0.03$ & $0.03 \pm 0.01$ \\
 & Phikon & $0.04 \pm 0.01$ & $0.05 \pm 0.02$ & $0.09 \pm 0.04$ & $0.11 \pm 0.03$ & $0.07 \pm 0.01$ \\
 & ResNet-50 & $0.18 \pm 0.04$ & $\mathbf{0.01 \pm 0.01}$ & $0.17 \pm 0.05$ & $0.17 \pm 0.04$ & $0.13 \pm 0.02$ \\
 & RetCCL & $0.03 \pm 0.02$ & $0.02 \pm 0.02$ & $0.03 \pm 0.04$ & $0.09 \pm 0.04$ & $0.04 \pm 0.02$ \\
 & Lunit-SwAV & $0.03 \pm 0.03$ & $0.05 \pm 0.03$ & $0.04 \pm 0.02$ & $0.05 \pm 0.03$ & $0.04 \pm 0.01$ \\
 & Swin & $0.20 \pm 0.04$ & $0.05 \pm 0.03$ & 

In [8]:
# Aggregate test AUROCS over seeds
t = test_aurocs.agg(["mean", "std"])
# means = t["mean"].unstack("target").unstack("model")
# stds = t["std"].unstack("target").unstack("model")

# formatted = means.map(lambda x: f"{x:.2f}") + " \\pm " + stds.map(lambda x: f"{x:.2f}")
# # Make best model bold
# best_mask = means == means.max(axis="index")
# formatted[best_mask] = "\\mathbf{" + formatted[best_mask] + "}"
# formatted = "$" + formatted + "$"
# formatted = formatted.stack("model")
# formatted = formatted.swaplevel("feature_extractor", "model").sort_index()
# formatted = formatted.rename(RENAME_MODELS, level="model")
# formatted = formatted.rename(RENAME_FEATURE_EXTRACTORS, level="feature_extractor")
# formatted.index.names = ["Model", "Feature extractor"]
# formatted.columns.names = ["Target"]
# print(formatted.to_latex(column_format="l" * len(formatted.index.levels) + "|" + "c" * n_targets + "|c", escape=False))

# print(results_to_latex(r))
t = (
    test_aurocs.droplevel(["train_dataset", "test_dataset", "seed"])
    .reset_index()
    .groupby(["target", "model", "feature_extractor"])
    .agg(["mean", "std"])
    .droplevel(0, axis="columns")
)
t.columns.names = ["stats"]
t = t.unstack("target").swaplevel("stats", "target", axis="columns").sort_index(axis="columns")
print(results_to_latex(t, goal="max"))

\begin{tabular}{ll|cccccc}
\toprule
 & Target & CDH1 & KRAS & MSI & PIK3CA & TP53 & subtype \\
Model & Feature extractor &  &  &  &  &  &  \\
\midrule
\multirow[t]{9}{*}{AttMIL} & Lunit-BT & $0.76 \pm 0.01$ & $nan \pm nan$ & $0.57 \pm 0.08$ & $0.51 \pm 0.02$ & $\mathbf{0.79 \pm 0.03}$ & $0.70 \pm 0.03$ \\
 & CTransPath & $0.78 \pm 0.04$ & $0.61 \pm 0.03$ & $0.83 \pm 0.06$ & $0.60 \pm 0.01$ & $0.78 \pm 0.02$ & $\mathbf{0.81 \pm 0.03}$ \\
 & Lunit-DINO & $\mathbf{0.79 \pm 0.01}$ & $nan \pm nan$ & $\mathbf{0.89 \pm 0.03}$ & $0.62 \pm 0.02$ & $0.77 \pm 0.03$ & $0.77 \pm 0.02$ \\
 & Phikon & $0.76 \pm 0.01$ & $0.64 \pm 0.03$ & $0.87 \pm 0.03$ & $0.59 \pm 0.02$ & $0.72 \pm 0.04$ & $0.70 \pm 0.02$ \\
 & ResNet-50 & $0.62 \pm 0.04$ & $0.54 \pm nan$ & $0.72 \pm 0.02$ & $\mathbf{0.63 \pm 0.01}$ & $0.64 \pm 0.06$ & $0.64 \pm 0.03$ \\
 & RetCCL & $0.77 \pm 0.02$ & $\mathbf{0.65 \pm nan}$ & $0.83 \pm 0.03$ & $0.62 \pm 0.02$ & $0.78 \pm 0.05$ & $0.73 \pm 0.03$ \\
 & Lunit-SwAV & $0.78 \pm 0.03$ & $n

In [9]:
augmentation

'Macenko_patchwise'